In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pylot as plt
def calculate_moments(RCIN): 
   
    rows,cols = RCIN.shape

    roll_input = np.zeros(rows); pitch_input = np.zeros(rows); plunge_input = np.zeros(rows); yaw_input = np.zeros(rows)
    t = np.zeros(rows)
    for i in range(0,rows):                                       # Fills input arrays for different motions
        t[i] = RCIN.iloc[i,0]
        roll_input[i] = RCIN.iloc[i,4]
        pitch_input[i] = RCIN.iloc[i,3]
        plunge_input[i] = RCIN.iloc[i,5]
        yaw_input[i] = RCIN.iloc[i,6]
    
    nominal_roll_input = roll_input[1]                            # Nominal inputs are found when the drone is in hover mode
    nominal_pitch_input = pitch_input[1]                          # which is seen at the beggining of our flight test.
    nominal_plunge_input = plunge_input[1]                       # Therefore initial values are used as nominal input
    nominal_yaw_input = yaw_input[1]
    
    max_PWM = 1684                                               # Max and Min used to noramlize the input vector.
    min_PWM = 364                                               # Determined by recording the input values at max and min
    PWM_diff = max_PWM - min_PWM                                  # throttle using no propellors
    
    µ_roll = np.zeros(len(roll_input))
    µ_pitch = np.zeros(len(pitch_input))
    µ_plunge = np.zeros(len(plunge_input))
    µ_yaw = np.zeros(len(yaw_input))
    
    for i in range(0,rows):
        µ_roll[i] = (nominal_roll_input - roll_input[i])/PWM_diff           # Uses formula (PWM_nom - PWM_exc)/PWM_max_min_diff
        µ_pitch[i] = (nominal_pitch_input  - pitch_input[i])/PWM_diff       # to solve for input vectors for the four motions
        µ_plunge[i] = (nominal_plunge_input - plunge_input[i])/PWM_diff
        µ_yaw[i] = (nominal_yaw_input - yaw_input[i])/PWM_diff
    
    plunge_mean = np.mean(µ_plunge)
    yaw_mean = np.mean(µ_yaw)
    for i in range(0,rows):    
        µ_plunge[i] = µ_plunge[i] - plunge_mean
        µ_yaw[i] = µ_yaw[i] - yaw_mean        

    else:
        return µ_roll, µ_pitch, µ_plunge, µ_yaw

def export_model_df(dataframe, dataframe_name):
    path_dir = 'Data_Exports/' + 'dji' + '/' + dataframe_name + '_output.csv' # Exports data if doesnt already exists
    dataframe.to_csv(path_dir)
    print(dataframe_name + " data has successfully been saved to an excel file.\n")

def dji_data_parser():
    #----------------------------------------------------------
    # Step 0: Import dji flight log
    #----------------------------------------------------------                                       
    dji = pd.DataFrame(pd.read_csv('Downloads/Mar-3rd-2021-02-47PM-Flight-M600.csv'))
    rows,cols = dji.shape
    #----------------------------------------------
    # Step 1: Parse timestamps
    #----------------------------------------------
    timestamp = []

    # Each row is 1 second of data in this situation
    time = dji.iloc[0,1]
    hrs = int(time[11:12])*60*60*1000
    mins = int(time[14:15])*60*1000
    sec = int(time[17:18])*1000
    ms = 0
    for i in range(0,rows):
        if i != 0:
            ms += 100
        timestamp.append(hrs + mins + sec + ms)
    print(timestamp)

    #-----------------------------------------------------------------------------------------------------------------
    # Step 2 - Obtain Derivatives for System Identification
    #-----------------------------------------------------------------------------------------------------------------
    # Obtain attitude angles and VD using NKF1 to take derivatives for angular acceleration and acceleration in z-axis   
    roll = np.zeros(rows) ; pitch = np.zeros(rows); yaw = np.zeros(rows); VD = np.zeros(rows)
    for i in range(0,rows):                       
        roll[i] = dji.iloc[i,24]                                  # rad
        pitch[i] = dji.iloc[i,23]                                 # rad
        yaw[i] = dji.iloc[i,22]                                   # rad
        VD[i] = dji.iloc[i,21]*0.44704                                   # m/s

    # First Derivatives (Angular Velocities and Acceleration in z-axis)
    dRdt = np.gradient(roll)*20                                  # rad/s
    dPdt = np.gradient(pitch)*20                                 # rad/s
    dYdt = np.gradient(yaw)*20                                   # rad/s
    dVDdt = np.gradient(VD)*20                                   # m/s^2

    # Second Derivatives (Angular Accelerations)
    dRdt2 = np.gradient(dRdt)*20                                 # rad/s^2
    dPdt2 = np.gradient(dPdt)*20                                 # rad/s^2
    dYdt2 = np.gradient(dYdt)*20                                 # rad/s^2

    # Obtain Translational Acceleration using Ground Speed from GPS data set
    v = np.zeros(rows) 
    alt = np.zeros(rows)
    for i in range(0,rows):
        v[i] = dji.iloc[i,9]*0.44704                                   # m/s
        alt[i] = dji.iloc[i,4]*0.3048               # m

    # First Derivative (Translational Acceleration)
    dvdt = np.gradient(v)*20                                     # m/s^2
    #-------------------------------------------------
    # Step 3: Gather data for different motion models
    #-------------------------------------------------
    # Already have attitiude angles, VD, and translational velocity from above
    # Gather the rest of position and velocity terms from NKF1

    rows,cols = dji.shape
    VN = np.zeros(rows); VE = np.zeros(rows)
    PN = np.zeros(rows); PE = np.zeros(rows); PD = np.zeros(rows)

    seconds_from_start = np.zeros(rows)


    for i in range(0,rows):
        seconds_from_start[i] = dji.iloc[i,0]*.001
        VN[i] = dji.iloc[i,20]
        VE[i] = dji.iloc[i,19]
        # ------------------------------------------------
        # Will be some trigonometry to obtain these values
        PN[i] = (dji.iloc[i,10]*0.3048)*math.cos(yaw[i])
        PE[i] = (dji.iloc[i,10]*0.3048)*math.sin(yaw[i])
        # ------------------------------------------------
        PD[i] = dji.iloc[i,4]*0.3048

    # First Derivatives (Translational Accelerations)
    dVNdt = np.gradient(VN) * 20                                 # m/s^2
    dVEdt = np.gradient(VE) * 20                                 # m/s^2

    # Calculating motion moments using RCIN and RCOU values
    µ_roll, µ_pitch, µ_plunge, µ_yaw = calculate_moments(dji) 
    
    plt.plot(roll,seconds_from_start)
    plt.show()
    plt.plot(pitch,seconds_from_start)
    plt.show()
    plt.plot(yaw,seconds_from_start)
    plt.show()
    plt.plot(VD,seconds_from_start)
    plt.show()
    
    for i in range

    # Already have angular rates from differentiation of attitude above
    #----------------------------------------------------------------------------------------------------------------- 
    # Step 5: Build dataframes for each motion model
    #-----------------------------------------------------------------------------------------------------------------
    # Motion Model 1: Roll - (Roll Acceleration, PE, Roll Angle(φ), VE, Roll Speed(p), µ_roll)
    #-----------------------------------------------------------------------------------------
    roll_model = pd.DataFrame()
    roll_model['Timestamp'] = timestamp
    roll_model['(d/dt)p'] = dRdt2
    roll_model['(d/dt)VE'] = dVEdt
    roll_model['PE'] = PE
    roll_model['Roll'] = roll
    roll_model['VE'] = VE
    roll_model['p'] = dRdt
    roll_model['Mu_roll'] = µ_roll
    roll_model['Flight Time (s)'] = seconds_from_start

    # Motion Model 2: Pitch - (Pitch Acceleration, PN, Pitch Angle(θ), VN, Pitch Speed(q), µ_pitch)
    #-----------------------------------------------------------------------------------------------
    pitch_model = pd.DataFrame()
    pitch_model['Timestamp'] = timestamp
    pitch_model['(d/dt)q'] = dPdt2
    pitch_model['(d/dt)VN'] = dVNdt
    pitch_model['PN'] = PN
    pitch_model['Pitch'] = pitch
    pitch_model['VN'] = VN
    pitch_model['q'] = dPdt
    pitch_model['Mu_pitch'] = µ_pitch
    pitch_model['Flight Time (s)'] = seconds_from_start

    # Motion Model 3: Plunge - (Vert Acceleration, PD, VD, µ_plunge)
    #---------------------------------------------------------------
    plunge_model = pd.DataFrame()
    plunge_model['Timestamp'] = timestamp
    plunge_model['(d/dt)VD'] = dVDdt
    plunge_model['PD'] = PD
    plunge_model['VD'] = VD
    plunge_model['Mu_plunge'] = µ_plunge
    plunge_model['Flight Time (s)'] = seconds_from_start

    # Motion Model 4: Yaw - (Yaw Acceleration, Yaw Angle(ψ), Translational Velocity(v), Yaw Speed(r), µ_yaw)
    #-------------------------------------------------------------------------------------------------------
    yaw_model = pd.DataFrame()
    yaw_model['Timestamp'] = timestamp
    yaw_model['(d/dt)r'] = dYdt2
    yaw_model['(d/dt)v'] = dvdt
    yaw_model['Yaw'] = yaw
    yaw_model['v'] = v
    yaw_model['r'] = dYdt
    yaw_model['Mu_yaw'] = µ_yaw
    yaw_model['Flight Time (s)'] = seconds_from_start

    # Full Model
    #-------------------------------------------------------------------------------------------------------
    full_model = pd.DataFrame()
    full_model['Timestamp'] = timestamp
    full_model['x'] = PN
    full_model['y'] = PE
    full_model['z'] = PD
    full_model['theta'] = pitch
    full_model['phi'] = roll
    full_model['psi'] = yaw
    full_model['u'] = VN
    full_model['v'] = VE
    full_model['w'] = VD
    full_model['q'] = dPdt
    full_model['p'] = dRdt
    full_model['r'] = dYdt
    full_model['Mplunge'] = µ_plunge
    full_model['Mpitch'] = µ_pitch
    full_model['Mroll'] = µ_roll
    full_model['Myaw'] = µ_yaw
    full_model['Alt'] = alt
    full_model['Flight Time (s)'] = seconds_from_start

    #----------------------------------------------------------------------------------------------------------------- 
    # Step 6: Export data frames into csv files
    #-----------------------------------------------------------------------------------------------------------------
    export_model_df(roll_model, "Roll_Motion_Model")
    export_model_df(pitch_model, "Pitch_Motion_Model")
    export_model_df(plunge_model, "Plunge_Motion_Model")
    export_model_df(yaw_model, "Yaw_Motion_Model") 
    export_model_df(full_model, "Full_Model")

In [2]:
dji_data_parser()

[7442000, 7442100, 7442200, 7442300, 7442400, 7442500, 7442600, 7442700, 7442800, 7442900, 7443000, 7443100, 7443200, 7443300, 7443400, 7443500, 7443600, 7443700, 7443800, 7443900, 7444000, 7444100, 7444200, 7444300, 7444400, 7444500, 7444600, 7444700, 7444800, 7444900, 7445000, 7445100, 7445200, 7445300, 7445400, 7445500, 7445600, 7445700, 7445800, 7445900, 7446000, 7446100, 7446200, 7446300, 7446400, 7446500, 7446600, 7446700, 7446800, 7446900, 7447000, 7447100, 7447200, 7447300, 7447400, 7447500, 7447600, 7447700, 7447800, 7447900, 7448000, 7448100, 7448200, 7448300, 7448400, 7448500, 7448600, 7448700, 7448800, 7448900, 7449000, 7449100, 7449200, 7449300, 7449400, 7449500, 7449600, 7449700, 7449800, 7449900, 7450000, 7450100, 7450200, 7450300, 7450400, 7450500, 7450600, 7450700, 7450800, 7450900, 7451000, 7451100, 7451200, 7451300, 7451400, 7451500, 7451600, 7451700, 7451800, 7451900, 7452000, 7452100, 7452200, 7452300, 7452400, 7452500, 7452600, 7452700, 7452800, 7452900, 7453000, 